# Python - Checking and Testing Code
## Learning Objectives

At the end of this lesson you will be able to:
 * Identify appropriate testing goals.
 * Run tests using the PyTest framework.
 * Create unit-tests.
 * Use Mocks.
 * Check code quality using flake8 and mypy.
 
  


## Testing software

Although we cannot prove our code is free of defects, or bugs, we can, and should, establish that it behaves as intended.

In a research environment it is often the case that there is no explicit specification for the software we create.

### System level testing

Once we have completed our software we should ensure that it works as intended.  This is referred to as - validation testing.  Typically this will involve taking a sample of input data and ensuring that the output of our software is as expected for the given input.

This *validation* testing will tell us in the overall system runs, and produces valid results.  Such tests should be repeated when changes are made to the software, to ensure the changes have not introduced errors.

### 